## ERP SCANR


.

In [1]:
## Imports

# Import custom code
from erpsc import *

# TEST IMPORTS
import requests
from bs4 import BeautifulSoup

In [2]:
## Things to search through - full set
erps = ['P50', 'P100', 'P200', 'P300', 'P3a', 'P3b', 'P600', 'N100', 'N170', 'N200',\
         'N270', 'N2pc', 'N400', 'MMN', 'LPC', 'CNV', 'ERN', 'ELAN', 'CPS', 'LRP', 'LDN']
cogs = ['language', 'memory', 'attention', 'motor', 'decision making', 'vision', \
         'auditory', 'emotion', 'categorization', 'reward', 'spatial']

In [4]:
# Small test set of words
erps = ['P50', 'P100']
cogs = ['language', 'memory']

## Co-Occurence of Term Analysis

.

In [5]:
# Initialize object for term count co-occurences. 
term_counts = ERPSC_Count()

# Set erp and cog lists as terms to use
term_counts.set_erps(erps)
term_counts.set_cogs(cogs)

In [6]:
# Scrape the co-occurence of terms data
term_counts.scrape_data()

In [7]:
# Check the most commonly associated COG term for each ERP
term_counts.check_erps()

For the  P50   the most common association is 	 memory     with 	 %01.39
For the  P100  the most common association is 	 memory     with 	 %02.44


In [8]:
# Check the most commonly associated ERP term for each COG
term_counts.check_cogs()

For  language             the strongest associated ERP is 	 P100  with 	 %00.60
For  memory               the strongest associated ERP is 	 P100  with 	 %02.44


In [9]:
# Check the terms with the most papers
term_counts.check_top()

The most studied ERP is  P50     with     8177 papers
The most studied COG is  memory  with   213047  papers


In [10]:
# Check how many papers were found for each term - ERPs
term_counts.check_counts('erp')

P50   -     8177
P100  -     2171


In [11]:
# Check how many papers were found for each term - COGs
term_counts.check_counts('cog')

language           -     135605
memory             -     213047


In [12]:
# Save pickle file of results
term_counts.save_pickle()

In [13]:
# Test load pickle
aa = load_pickle_counts()

In [14]:
aa.check_counts('cog')

language           -     135605
memory             -     213047


## WORD ANALYSIS

.

In [ ]:
test = ERPSC_Words()

In [17]:
# Test Code

#page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=word&term=“N270”AND”Language”')
page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=word&term=“P50”&retmax=50')

page_soup = BeautifulSoup(page.content)

counts = page_soup.find_all('count')

for i in range(0, len(counts)):
    count = counts[i]
    ext = count.text
    print int(ext)

8177
8177


In [18]:
page_soup

<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "http://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<html><body><esearchresult><count>8177</count><retmax>50</retmax><retstart>0</retstart><idlist>
<id>27315458</id>
<id>27308511</id>
<id>27302421</id>
<id>27295376</id>
<id>27290699</id>
<id>27287810</id>
<id>27281270</id>
<id>27279585</id>
<id>27271273</id>
<id>27268961</id>
<id>27264783</id>
<id>27264614</id>
<id>27262063</id>
<id>27256734</id>
<id>27256730</id>
<id>27252578</id>
<id>27250735</id>
<id>27243897</id>
<id>27242078</id>
<id>27242072</id>
<id>27238796</id>
<id>27234131</id>
<id>27217271</id>
<id>27216047</id>
<id>27198178</id>
<id>27197825</id>
<id>27197812</id>
<id>27194765</id>
<id>27190325</id>
<id>27183665</id>
<id>27180104</id>
<id>27178864</id>
<id>27178167</id>
<id>27176922</id>
<id>27168850</id>
<id>27168061</id>
<id>27162449</id>
<id>27161000</id>
<id>27147748</id>
<id>27145282</id>
<id>27141897</id>
<id>27139571</id>
<id>27138602</id

In [19]:
aa = page_soup.find_all('id')

In [20]:
aa[1].text

u'27308511'

In [22]:
len(aa)

50

In [24]:
page_1 = requests.get('http://www.ncbi.nlm.nih.gov/pubmed/' + aa[0].text)
page_1_soup = BeautifulSoup(page_1.content)

In [25]:
page_1_soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head xmlns:xi="http://www.w3.org/2001/XInclude"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- meta -->
<meta content="index,nofollow,noarchive" name="robots"/><meta content="http://www.ncbi.nlm.nih.gov/coreutils/img/pubmed256blue.png" property="og:image"/><meta content="https://www.ncbi.nlm.nih.gov/coreutils/img/pubmed256blue.png" property="og:image:secure_url"/>
<meta content="entrez" name="ncbi_app"/><meta content="pubmed" name="ncbi_db"/><meta content="abstract" name="ncbi_report"/><meta content="html" name="ncbi_format"/><meta content="20" name="ncbi_pagesize"/><meta content="default" name="ncbi_sortorder"/><meta content="1" name="ncbi_pageno"/><meta content="1" name="ncbi_resultcount"/><meta content="retrieve" name="ncbi_op"/><meta content="abstract" name="ncbi_pdid"/><meta content="CE887D

In [31]:
title = page_1_soup.find('title')
print title

abtract = page_1_soup.find('abstracttext')

<title>Association of catechol-O-methyltransferase Val108/158 Met genetic polymorphism with schizophrenia, P50 sensory gating, and negative symptoms in a ...  - PubMed - NCBI
</title>


In [32]:
aa = page_1_soup.find_all('h1')

In [33]:
aa

[<h1 class="img_logo"><a class="pmlogo offscreen" href="/pubmed" title="PubMed">PubMed</a></h1>,
 <h1>Association of catechol-O-methyltransferase Val108/158 Met genetic polymorphism with schizophrenia, P50 sensory gating, and negative symptoms in a Chinese population.</h1>]